<center> <h2>Universidad de Sonora </h2></center>
<center> <h2>Ariel David López Cota </h2></center>
<center> <h2>Estadistica MCD </h2></center>
<!-- <center> <h2>  </h2></center> -->
<center> 

![](unison.png)

</center>






<center> Introducción </center>
<p> Esta presentación es para un caso de estudio de biociencia de Bella Anahi, aqui uncluire codigo asi como explicación de lo que realize.</p>
<p> Me enfoqué en la realización de una visualización de a traves del tiempo, apliqué 3 metodos de cluster, espero sean de utilidad.</p>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

#### Lectura de los datos
Base de datos proporcionados por Bella Flor

In [2]:
# Read Excel file
df_TB4 = pd.read_excel("TB4.xlsx")

Creacion de una tabla, con 3 columnas, las especies/familia/genero estaran en una columna

In [3]:
# We will create a new table categorizing the data, to reduce the columns convetirng them to a "Especie" column
df_TB4_Test = df_TB4.copy()
df_Ordered = pd.DataFrame(columns=["Profundidad", "Element", "Valor"])

list_columns = df_TB4_Test.columns[df_TB4_Test.columns != "Profundidad"]

for col in list_columns:
    for nrow in range(0,len(df_TB4_Test.index)):
        specie = col
        profundidad = df_TB4_Test["Profundidad"].iloc[nrow]
        valor = df_TB4_Test[col].iloc[nrow]
        # Add a new row
        df_Ordered.loc[len(df_Ordered.index)] = [profundidad, specie, valor] 

df_Ordered

,Profundidad,Element,Valor
0,5.0,Abies,0.000
1,10.0,Abies,0.000
2,15.0,Abies,0.000
3,18.0,Abies,0.000
4,21.0,Abies,0.000
...,...,...,...
12982,322.5,Indeterminables,3.125
12983,325.0,Indeterminables,3.667
12984,327.5,Indeterminables,1.242
12985,330.0,Indeterminables,0.000


In [99]:
# CLEAN THIS VALUES
list_dif = list(set(df_Ordered.Element.unique()) - set(df_clasification_clean.Element.unique()))
list_dif

['Indeterminables',
 'Asplenium ',
 'Euphorbia ',
 'Indeterminados',
 'Poaceae >60µm',
 'Tribulus ',
 'Kallstroemia ',
 'Unknown trilete']

## Line charts

Con base a los datos ordenados, ahora somos capaces de crear un grafico que nos permita visualizar de manera clara y sencilla, la proporcion que exsitia atraves del tiempo

In [6]:
fig = go.Figure()

for specie in df_Ordered["Element"].unique():
    if specie == "Pinus":
        fig.add_trace(go.Scatter(x=df_Ordered[df_Ordered["Element"] == specie]["Profundidad"], y=df_Ordered[df_Ordered["Element"] == specie]["Valor"], name=specie,
                         line = dict(color='green', width=4)))
    else:
        fig.add_trace(go.Scatter(x=df_Ordered[df_Ordered["Element"] == specie]["Profundidad"], y=df_Ordered[df_Ordered["Element"] == specie]["Valor"], name=specie,
                         line = dict(color='gray', width=1)))

fig.update_layout(title='Cantidad de proporcionn',
                   xaxis_title='Profundidad',
                   yaxis_title='Valor')
fig.show()

##
Dejamos esta grafico para que el lector pueda realizar comparaciones 

In [40]:
# import plotly.graph_objects as go
fig = go.Figure()

for specie in df_Ordered["Element"].unique():
    if specie == "Pinus":
        fig.add_trace(go.Scatter(x=df_Ordered[df_Ordered["Element"] == specie]["Profundidad"], y=df_Ordered[df_Ordered["Element"] == specie]["Valor"], name=specie,
                         line = dict(color='green', width=4)))
    elif specie == "Asteroideae":
        fig.add_trace(go.Scatter(x=df_Ordered[df_Ordered["Element"] == specie]["Profundidad"], y=df_Ordered[df_Ordered["Element"] == specie]["Valor"], name=specie,
                         line = dict(color='red', width=4)))
    else:
        fig.add_trace(go.Scatter(x=df_Ordered[df_Ordered["Element"] == specie]["Profundidad"], y=df_Ordered[df_Ordered["Element"] == specie]["Valor"], name=specie,
                         line = dict(color='gray', width=1)))

fig.update_layout(title='Cantidad de proporcion',
                   xaxis_title='Profundidad',
                   yaxis_title='Prcentaje')
fig.show()

<p>En base a los datos y sus propiedades he decidido realizar clusters, para una visualización mas clara con menos objetos en el grafico.</p>
<p>¿Que criterios se tomaran para definir los clusters?</p>
<p>Se segregaran por Genero y Familia, con la ayuda</p>

---

### Preambulo
Una de las problematicas del problema planteado por nuestro "Cliente", es que los elemntos que proporciona como una propiedad (un columna por cada dato), no son homologos uno del otro, por lo cual considero que no es correcto hacer comparaciones uno con el otro. 

<h4>¿Cual es problema con estas propiedades?</h4>
<p>El cliente nos comenta que a veces nos posible identificar la especie del polen, pero si su genero o su familia, entonces tenemos 3 tipos de elementos, los cuales si no hacemos una segregación (opinion personal) podriamos mostrar resultados sesgados y/o erroneos.</p>
<p>Para explicar mejor esta problematica abajo expongo un ejemplo en una imagen, como pueden ver observar la especie "Cardinalis cardinales" se encuentra en todas las "etapas" de la taxonomia, es decir la familia Fringilidae contiene a Cardinalis cardinales asi como tambien el genero Cardinales lo contiene. Entonces tomando esto en cuenta, pudieramos encontrarnos que una especie no este presente o en bastante proprcion con lo datos, pero realmente si pudiera estar bastante presente poruqe tal vez este incluido en su genero o su familia.</p>

![](taxonomia-animal.jpg)

https://www.areaciencias.com/biologia/taxonomia-clasificacion-de-los-seres-vivos/

<h4>¿Como pudieramos resolver este problematica o realizar una representación mas precisa?</h4>

Para este caso particualar, la solucion la he dividido en 4 pasos:
- Clasificación de los elementos
   - Los elementos seran clasificados por su tipo (Especie, Genero y Familia)
- Limpieza y carga de la clasificación
- Creación de clusters
- Visualización de clusters
---

<h3> 1 - Clasificación de los elementos </h3>

Debido al poco tiempo, la cantidad de datos y mi escazo conocimiento del tema, he recurrido a una herramienta de Inteligencia Artificial para que me realice esta clasificación de manera rapida y eficaz.
Como podran ver en la imagen, he logrado que me clasifique los elementos en una tabla, para ello tuve que realizar varias preguntas y pulirlas para lograr el resultado deseado.

![](ChatGPT_Classification.png)

<h4>¿Esta clasificación es correcta?</h4>
<p>Existe una alta probabilidad que todos resultados no sean correctos, pero, claro esta que dejamos al lector los datos, para que si desea una información mas precisa, pueda modificarlos.</p>



---

<h3> 2 - Limpieza y Carga de la clasificación </h3>
<p>Ya con el resultado en una tabla procedere a realizar una limpieza de los datos, habiendo extraido la tabla y pasandolo a un archivo csv.</p>

In [4]:
# Limpieza de los datos
df_clasification = pd.read_csv('Clasification.csv', encoding = "utf-8")
df_clasification

,Element,Type,Species,Genus,Family
0,Abies,Species,Abies,Abies,Pinaceae
1,Acacia,Species,Acacia,Acacia,Fabaceae
2,Agave,Species,Agave,Agave,Asparagaceae
3,Alnus oblongifolia,Species,Alnus oblongifolia,Alnus,Betulaceae
4,Anisacanthus type,Species,Anisacanthus,Anisacanthus,Acanthaceae
...,...,...,...,...,...
112,Pellaea,Genus,-,Pellaea,Pteridaceae
113,Spore trilete without perine,Species,-,Unknown,Unknown
114,Unknown trilete,Species,-,Unknown,Unknown
115,Indeterminados,Species,-,Unknown,Unknown


In [39]:
# Familia desconocida, elementos no parecen ser ninguno
df_clasification[(df_clasification['Species'] == '-') & (df_clasification['Type'] == 'Species')]

,Element,Type,Species,Genus,Family
114,Unknown trilete,Species,-,Unknown,Unknown
115,Indeterminados,Species,-,Unknown,Unknown
116,Indeterminables,Species,-,Unknown,Unknown


In [5]:
# Corregiremos 4 valores (Cf.Populus, Reticulate trilete, )
df_clasification.loc[14].Species = df_clasification.loc[14].Element
df_clasification.loc[81].Type = 'Family'
df_clasification.loc[110].Species = df_clasification.loc[110].Element
df_clasification.loc[113].Species = df_clasification.loc[113].Element
# Quitaremos los elementos desconocidos, no creo que sea de utilidad conocer la proporcion de elementos desconocidos (a reserva que le lector lo desea, puede dejarlos)
df_clasification_clean = df_clasification[df_clasification.index < 114].copy()
df_clasification_clean

,Element,Type,Species,Genus,Family
0,Abies,Species,Abies,Abies,Pinaceae
1,Acacia,Species,Acacia,Acacia,Fabaceae
2,Agave,Species,Agave,Agave,Asparagaceae
3,Alnus oblongifolia,Species,Alnus oblongifolia,Alnus,Betulaceae
4,Anisacanthus type,Species,Anisacanthus,Anisacanthus,Acanthaceae
...,...,...,...,...,...
109,Osmunda,Genus,-,Osmunda,Osmundaceae
110,Reticulate trilete,Species,Reticulate trilete,Unknown,Unknown
111,Selaginella,Genus,-,Selaginella,Selaginellaceae
112,Pellaea,Genus,-,Pellaea,Pteridaceae


---

### Clasificacion por familia
A continuación crearemos un nuevo dataframe (tabla) para almacenar los valores por cada profundiad por familia, siendo la suma de los valores de todas las especies y generos que pertenezcan a la familia.

In [6]:
df_Ordered_Complete = df_Ordered.merge(df_clasification_clean, on='Element')
df_Ordered_Complete

# Creación de un nuevo dataframe para albergar solo las familias
df_elements_by_famliy = pd.DataFrame(columns=["Profundidad", "Family", "Valor"])
for fam in df_Ordered_Complete["Family"].unique():
    for prof in df_Ordered_Complete["Profundidad"].unique():
        sumFamElements = sum(df_Ordered_Complete[(df_Ordered_Complete["Family"] == fam) & (df_Ordered_Complete["Profundidad"] == prof)]["Valor"])
        df_elements_by_famliy.loc[len(df_elements_by_famliy.index)] = [prof, fam, sumFamElements] 


In [8]:
fig = go.Figure()
for specie in df_elements_by_famliy["Family"].unique():
    if specie == "Pinaceae":
        fig.add_trace(go.Scatter(x=df_elements_by_famliy[df_elements_by_famliy["Family"] == specie]["Profundidad"], y=df_elements_by_famliy[df_elements_by_famliy["Family"] == specie]["Valor"], name=specie,
                         line = dict(color='green', width=4)))
    else:
        fig.add_trace(go.Scatter(x=df_elements_by_famliy[df_elements_by_famliy["Family"] == specie]["Profundidad"], y=df_elements_by_famliy[df_elements_by_famliy["Family"] == specie]["Valor"], name=specie,
                         line = dict(color='gray', width=1)))

fig.update_layout(title='Cantidad de proporcion por Familia',
                   xaxis_title='Profundidad',
                   yaxis_title='Porcentaje')
fig.show()

### __Clasificacion tabla Bella__

<p>
En esta parte se realiza una clasificación de los elementos de la tabla original de los datos, se agregan valores : Tipo, Especie, Genero, Familia y Tipo_Ecosistema, estos son generados a partir de la tabla proporcionada por Bella, donde nos indica que clasificacion es cada elemento.
<p>

In [206]:
# Clasificación con tabla Bella
df_clasification_bella = pd.read_excel("Tabla de tipos funcionales y familias.xlsx")
df_Ordered_Bella = df_Ordered.copy()
df_Ordered_Bella["Tipo"] = ""
df_Ordered_Bella["Especie"] = ""
df_Ordered_Bella["Genero"] = ""
df_Ordered_Bella["Familia"] = ""
df_Ordered_Bella["Tipo_Ecosistema"] = ""

for elem in df_Ordered_Bella["Element"].unique():
    if len(df_clasification_bella[df_clasification_bella["Especie"] == elem]) > 0:
        df_Ordered_Bella.loc[df_Ordered_Bella["Element"] == elem, "Tipo"] = "Especie"
        df_Ordered_Bella.loc[df_Ordered_Bella["Element"] == elem, "Especie"] = elem
        df_Ordered_Bella.loc[df_Ordered_Bella["Element"] == elem, "Genero"] = df_clasification_bella[df_clasification_bella["Especie"] == elem]["Genero "].values[0]
        df_Ordered_Bella.loc[df_Ordered_Bella["Element"] == elem, "Familia"] = df_clasification_bella[df_clasification_bella["Especie"] == elem]["Familia"].values[0]
        df_Ordered_Bella.loc[df_Ordered_Bella["Element"] == elem, "Tipo_Ecosistema"] = df_clasification_bella[df_clasification_bella["Especie"] == elem]["Tipo"].values[0]

    elif len(df_clasification_bella[df_clasification_bella["Genero "] == elem]) > 0:
        df_Ordered_Bella.loc[df_Ordered_Bella["Element"] == elem, "Tipo"] = "Genero"
        df_Ordered_Bella.loc[df_Ordered_Bella["Element"] == elem, "Genero"] = elem
        df_Ordered_Bella.loc[df_Ordered_Bella["Element"] == elem, "Familia"] = df_clasification_bella[df_clasification_bella["Genero "] == elem]["Familia"].values[0]
        df_Ordered_Bella.loc[df_Ordered_Bella["Element"] == elem, "Tipo_Ecosistema"] = df_clasification_bella[df_clasification_bella["Genero "] == elem]["Tipo"].values[0]
    elif len(df_clasification_bella[df_clasification_bella["Familia"] == elem]) > 0:
        df_Ordered_Bella.loc[df_Ordered_Bella["Element"] == elem, "Tipo"] = "Familia"
        df_Ordered_Bella.loc[df_Ordered_Bella["Element"] == elem, "Familia"] = elem
        df_Ordered_Bella.loc[df_Ordered_Bella["Element"] == elem, "Tipo_Ecosistema"] = df_clasification_bella[df_clasification_bella["Familia"] == elem]["Tipo"].values[0]

# Creación de un nuevo dataframe para albergar solo las familias
df_Ordered_Bella_Family = pd.DataFrame(columns=["Profundidad", "Family", "Valor", "Tipo_Ecosistema"])
for fam in df_Ordered_Bella["Familia"].unique():
    for prof in df_Ordered_Bella["Profundidad"].unique():
        sumFamElements = sum(df_Ordered_Bella[(df_Ordered_Bella["Familia"] == fam) & (df_Ordered_Bella["Profundidad"] == prof)]["Valor"])
        eco = df_Ordered_Bella[(df_Ordered_Bella["Familia"] == fam) & (df_Ordered_Bella["Profundidad"] == prof)]["Tipo_Ecosistema"].values[0]
        df_Ordered_Bella_Family.loc[len(df_Ordered_Bella_Family.index)] = [prof, fam, sumFamElements, eco ]

df_Ordered_Bella_Ecosistema = pd.DataFrame(columns=["Profundidad", "Valor", "Tipo_Ecosistema"])
for eco in df_Ordered_Bella["Tipo_Ecosistema"].unique():
    for prof in df_Ordered_Bella["Profundidad"].unique():
        sumFamElements = sum(df_Ordered_Bella[(df_Ordered_Bella["Tipo_Ecosistema"] == eco) & (df_Ordered_Bella["Profundidad"] == prof)]["Valor"])       
        df_Ordered_Bella_Ecosistema.loc[len(df_Ordered_Bella_Ecosistema.index)] = [prof, sumFamElements, eco] 

In [212]:
fig = px.line(df_Ordered_Bella_Family, x="Profundidad", y="Valor", color="Family", title="Proporcion de valores por Familia")
fig.show()

In [209]:
fig = px.line(df_Ordered_Bella_Ecosistema, x="Profundidad", y="Valor", color="Tipo_Ecosistema", title="Proporcion de valores por tipo")
fig.show()

### Clasificacion por Genero

In [210]:
# Creación de un nuevo dataframe para albergar solo los generos
df_elements_by_genus = pd.DataFrame(columns=["Profundidad", "Genus", "Valor"])
for gen in df_Ordered_Complete["Genus"].unique():
    for prof in df_Ordered_Complete["Profundidad"].unique():
        sumGenElements = sum(df_Ordered_Complete[(df_Ordered_Complete["Genus"] == gen) & (df_Ordered_Complete["Profundidad"] == prof)]["Valor"])
        df_elements_by_genus.loc[len(df_elements_by_genus.index)] = [prof, gen, sumGenElements] 

In [213]:
fig = px.line(df_elements_by_genus, x="Profundidad", y="Valor", color='Genus')
fig.show()

---

<h2> 3 - Creación de Clusters </h2>

#### Premicia

<p> Durante la primer intento de creación de cluster tuve un gran error, la forma en que realice los cluster no fue la adecuada, ya que tenia diferentes familia que pertenecian a diferentes clusters en diferentes tiempos, lo cual no es correcto.  </p>
<p>
Para resolver esta problematica, lo que hicé, fué crear una tabla, donde las profundidades (años) sean columnas, y las familias/Generos sean renglones, asi el cluster sera por cada familia a lo largo del tiempo.
</p>

In [ ]:
# Creacion de tabla donde profundidades son propiedades (Columnas) - Familia
df_profundidades_family = pd.DataFrame()
countCol = 0
for prof in df_elements_by_famliy["Profundidad"].unique():
    df_profundidades_family.insert(countCol, str(prof), list(df_elements_by_famliy[df_elements_by_famliy["Profundidad"] == prof]["Valor"]), True)
    countCol = countCol + 1
df_profundidades_family.insert(0, 'Family', list(df_elements_by_famliy["Family"].unique()), True)
# Create Numetic family number to make cluster
df_profundidades_family.insert(0, 'Family_Numeric', pd.factorize(df_profundidades_family["Family"])[0], True)
# df_features_1 = df_features_1.drop(columns='Family')

# Creacion de tabla donde profundidades son propiedades (Columnas) - Genero
df_profundidades_genero = pd.DataFrame()
countCol = 0
for prof in df_elements_by_genus["Profundidad"].unique():
    df_profundidades_genero.insert(countCol, str(prof), list(df_elements_by_genus[df_elements_by_genus["Profundidad"] == prof]["Valor"]), True)
    countCol = countCol + 1
df_profundidades_genero.insert(0, 'Genus', list(df_elements_by_genus["Genus"].unique()), True)
# Create Numetic family number to make cluster
df_profundidades_genero.insert(0, 'Genus_Numeric', pd.factorize(df_profundidades_genero["Genus"])[0], True)
# df_features_1 = df_features_1.drop(columns='Family')

In [81]:
df_profundidades_family

,Family_Numeric,Family,5.0,10.0,15.0,18.0,21.0,24.0,27.0,30.0,...,310.0,312.5,315.0,317.5,320.0,322.5,325.0,327.5,330.0,331.0
0,0,Pinaceae,5.508,3.665,3.371,0.000,1.724,2.108,0.000,0.0,...,32.458,35.310,50.985,51.891,49.877,53.693,52.000,61.181,67.481,56.908
1,1,Fabaceae,15.254,20.943,11.798,1.667,0.000,0.301,0.938,0.0,...,0.000,0.000,0.246,0.000,0.000,0.284,0.000,0.000,0.000,0.000
2,2,Asparagaceae,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,3,Betulaceae,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,...,0.000,0.565,0.985,0.811,0.246,0.568,0.667,0.311,0.244,0.468
4,4,Acanthaceae,0.000,0.000,0.562,0.000,0.000,0.000,0.625,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,57,Lygodiaceae,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,...,5.728,0.000,2.217,0.000,1.474,0.000,0.000,0.000,1.711,0.000
58,58,Lycopodiaceae,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,...,0.000,0.000,0.000,0.811,0.000,0.000,0.000,0.000,0.000,0.000
59,59,Osmundaceae,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.234
60,60,Unknown,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,...,0.716,3.954,0.985,4.594,0.737,1.988,1.333,0.000,0.244,0.468


### Metodo 1 -  Agglomerative Clustering 

<p>En resumen, el clustering aglomerativo es un algoritmo que fusiona iterativamente los grupos más similares para construir una estructura jerárquica de grupos. Permite explorar diferentes niveles de granularidad en la formación de grupos y proporciona información visual a través del dendrograma.</p>

Bibliografía:

- Jain, A. K., Murty, M. N., & Flynn, P. J. (1999). Data clustering: a review. ACM computing surveys (CSUR), 31(3), 264-323.
- Everitt, B. S., Landau, S., Leese, M., & Stahl, D. (2011). Cluster analysis. John Wiley & Sons.
- Mullner, D. (2011). Modern hierarchical, agglomerative clustering algorithms. arXiv preprint arXiv:1109.2378.

<p> https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html</p>

In [82]:
# Removemos datos no numericos
df_profundidades_family_agg_15 = df_profundidades_family.copy().drop(columns='Family')

# Instanciar el modelo de AgglomerativeClustering con el numero deseado de clusters
agg_clustering = AgglomerativeClustering(n_clusters=15)
# Fit el modelo con los datos (solo numericos)
agg_clustering.fit(df_profundidades_family_agg_15)
# Add the cluster labels to the dataframe
df_profundidades_family_agg_15['Cluster'] = agg_clustering.labels_
df_profundidades_family_agg_15.insert(0, 'Family', list(df_profundidades_family["Family"]), True)

# --- 31 Cluster
# Removemos datos no numericos
df_profundidades_family_agg_31 = df_profundidades_family.copy().drop(columns='Family')

# Instanciar el modelo de AgglomerativeClustering con el numero deseado de clusters
agg_clustering = AgglomerativeClustering(n_clusters=31)
# Fit el modelo con los datos (solo numericos)
agg_clustering.fit(df_profundidades_family_agg_31)
# Add the cluster labels to the dataframe
df_profundidades_family_agg_31['Cluster'] = agg_clustering.labels_
df_profundidades_family_agg_31.insert(0, 'Family', list(df_profundidades_family["Family"]), True)

# Genero
# -------------
# Removemos datos no numericos
df_profundidades_genero_agg_15 = df_profundidades_genero.copy().drop(columns='Genus')

# Instanciar el modelo de AgglomerativeClustering con el numero deseado de clusters
agg_clustering = AgglomerativeClustering(n_clusters=15)
# Fit el modelo con los datos (solo numericos)
agg_clustering.fit(df_profundidades_genero_agg_15)
# Add the cluster labels to the dataframe
df_profundidades_genero_agg_15['Cluster'] = agg_clustering.labels_
df_profundidades_genero_agg_15.insert(0, 'Genus', list(df_profundidades_genero["Genus"]), True)
# -------------
# Removemos datos no numericos
df_profundidades_genero_agg_31 = df_profundidades_genero.copy().drop(columns='Genus')

# Instanciar el modelo de AgglomerativeClustering con el numero deseado de clusters
agg_clustering = AgglomerativeClustering(n_clusters=31)
# Fit el modelo con los datos (solo numericos)
agg_clustering.fit(df_profundidades_genero_agg_31)
# Add the cluster labels to the dataframe
df_profundidades_genero_agg_31['Cluster'] = agg_clustering.labels_
df_profundidades_genero_agg_31.insert(0, 'Genus', list(df_profundidades_genero["Genus"]), True)

<p> Muy bien, ahora ya tenemos nuestros cluster por cada familia y genero, ahora pasaremos esta información de cluster a nuestro dataframe ordenado para graficas.</p>

In [83]:
# Familia 
df_cluster_agg_15_fam = df_elements_by_famliy.copy()
list_Cluster = []
for fam in df_cluster_agg_15_fam["Family"]:
    cluster = df_profundidades_family_agg_15[df_profundidades_family_agg_15["Family"] == fam]["Cluster"].values[0]
    list_Cluster.append(cluster)
df_cluster_agg_15_fam["Cluster"] = list_Cluster

df_cluster_agg_31_fam = df_elements_by_famliy.copy()
list_Cluster = []
for fam in df_cluster_agg_31_fam["Family"]:
    cluster = df_profundidades_family_agg_31[df_profundidades_family_agg_31["Family"] == fam]["Cluster"].values[0]
    list_Cluster.append(cluster)
df_cluster_agg_31_fam["Cluster"] = list_Cluster

# Genero
df_cluster_agg_15_genus = df_elements_by_genus.copy()
list_Cluster = []
for fam in df_cluster_agg_15_genus["Genus"]:
    cluster = df_profundidades_genero_agg_15[df_profundidades_genero_agg_15["Genus"] == fam]["Cluster"].values[0]
    list_Cluster.append(cluster)
df_cluster_agg_15_genus["Cluster"] = list_Cluster

df_cluster_agg_31_genus = df_elements_by_genus.copy()
list_Cluster = []
for fam in df_cluster_agg_31_genus["Genus"]:
    cluster = df_profundidades_genero_agg_31[df_profundidades_genero_agg_31["Genus"] == fam]["Cluster"].values[0]
    list_Cluster.append(cluster)
df_cluster_agg_31_genus["Cluster"] = list_Cluster

Tablas para mostrar a que cluster pertenece cada familia

In [68]:
pd.set_option('display.max_colwidth', None)

df_Cluster_Families_15 = pd.DataFrame(columns=["Cluster", "Familias"])
for clus in df_cluster_agg_15_fam["Cluster"].unique():
    famsList = ""
    for fam in df_cluster_agg_15_fam[df_cluster_agg_15_fam["Cluster"] == clus]["Family"].unique():
        famsList = famsList + ", " + fam
    df_Cluster_Families_15.loc[len(df_Cluster_Families_15.index)] = [clus, famsList] 

df_Cluster_Families_15 = df_Cluster_Families_15.sort_values(by="Cluster")

df_Cluster_Families_31 = pd.DataFrame(columns=["Cluster", "Familias"])
for clus in df_cluster_agg_31_fam["Cluster"].unique():
    famsList = ""
    for fam in df_cluster_agg_31_fam[df_cluster_agg_31_fam["Cluster"] == clus]["Family"].unique():
        famsList = famsList + ", " + fam
    df_Cluster_Families_31.loc[len(df_Cluster_Families_31.index)] = [clus, famsList] 

df_Cluster_Families_31 = df_Cluster_Families_31.sort_values(by="Cluster")

df_Cluster_Genero_15 = pd.DataFrame(columns=["Cluster", "Genero"])
for clus in df_cluster_agg_15_genus["Cluster"].unique():
    famsList = ""
    for fam in df_cluster_agg_15_genus[df_cluster_agg_15_genus["Cluster"] == clus]["Genus"].unique():
        famsList = famsList + ", " + fam
    df_Cluster_Genero_15.loc[len(df_Cluster_Genero_15.index)] = [clus, famsList] 

df_Cluster_Genero_15 = df_Cluster_Genero_15.sort_values(by="Cluster")

df_Cluster_Genero_31 = pd.DataFrame(columns=["Cluster", "Genero"])
for clus in df_cluster_agg_15_genus["Cluster"].unique():
    famsList = ""
    for fam in df_cluster_agg_15_genus[df_cluster_agg_15_genus["Cluster"] == clus]["Genus"].unique():
        famsList = famsList + ", " + fam
    df_Cluster_Genero_31.loc[len(df_Cluster_Genero_31.index)] = [clus, famsList] 

df_Cluster_Genero_31 = df_Cluster_Genero_31.sort_values(by="Cluster")

### Method 2 - Kmeans

<p>Clustering K-means es un algoritmo que agrupa datos en función de su similitud. Funciona dividiendo el conjunto de datos en un número predeterminado de grupos, llamados "k", donde "k" representa el número de centroides que se utilizarán en el algoritmo.</p>

<p>El algoritmo K-means funciona de la siguiente manera:</p>

1. Selecciona aleatoriamente "k" puntos del conjunto de datos como centroides iniciales.
2. Asigna cada muestra del conjunto de datos al centroide más cercano basándose en una medida de distancia, comúnmente la distancia euclidiana.
3. Recalcula los centroides de cada grupo tomando el promedio de todas las muestras asignadas a ese grupo.
4. Repite los pasos 2 y 3 hasta que los centroides ya no cambien o se alcance un criterio de convergencia predefinido.



https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

In [66]:
# Removemos datos no numericos
df_profundidades_family_kmeans_15 = df_profundidades_family.copy().drop(columns='Family')

# Instanciar el modelo de KMeans con el numero deseado de clusters
kmeans = KMeans(n_clusters=15)
# Fit el modelo con los datos (solo numericos)
kmeans.fit(df_profundidades_family_kmeans_15)
# Add the cluster labels to the dataframe
df_profundidades_family_kmeans_15['Cluster'] = kmeans.labels_
df_profundidades_family_kmeans_15.insert(0, 'Family', list(df_profundidades_family["Family"]), True)

# --- 31 Cluster
# Removemos datos no numericos
df_profundidades_family_kmeans_31 = df_profundidades_family.copy().drop(columns='Family')

# Instanciar el modelo de AgglomerativeClustering con el numero deseado de clusters
kmeans = KMeans(n_clusters=31)
# Fit el modelo con los datos (solo numericos)
kmeans.fit(df_profundidades_family_kmeans_31)
# Add the cluster labels to the dataframe
df_profundidades_family_kmeans_31['Cluster'] = kmeans.labels_
df_profundidades_family_kmeans_31.insert(0, 'Family', list(df_profundidades_family["Family"]), True)

# Genero
# -------------
# Removemos datos no numericos
df_profundidades_genero_kmeans_15 = df_profundidades_genero.copy().drop(columns='Genus')

# Instanciar el modelo de AgglomerativeClustering con el numero deseado de clusters
kmeans = KMeans(n_clusters=15)
# Fit el modelo con los datos (solo numericos)
kmeans.fit(df_profundidades_genero_kmeans_15)
# Add the cluster labels to the dataframe
df_profundidades_genero_kmeans_15['Cluster'] = kmeans.labels_
df_profundidades_genero_kmeans_15.insert(0, 'Genus', list(df_profundidades_genero["Genus"]), True)
# -------------
# Removemos datos no numericos
df_profundidades_genero_kmeans_31 = df_profundidades_genero.copy().drop(columns='Genus')

# Instanciar el modelo de AgglomerativeClustering con el numero deseado de clusters
kmeans = KMeans(n_clusters=31)
# Fit el modelo con los datos (solo numericos)
kmeans.fit(df_profundidades_genero_kmeans_31)
# Add the cluster labels to the dataframe
df_profundidades_genero_kmeans_31['Cluster'] = kmeans.labels_
df_profundidades_genero_kmeans_31.insert(0, 'Genus', list(df_profundidades_genero["Genus"]), True)

In [85]:
# Familia 
df_cluster_kmeans_15_fam = df_elements_by_famliy.copy()
list_Cluster = []
for fam in df_cluster_kmeans_15_fam["Family"]:
    cluster = df_profundidades_family_kmeans_15[df_profundidades_family_kmeans_15["Family"] == fam]["Cluster"].values[0]
    list_Cluster.append(cluster)
df_cluster_kmeans_15_fam["Cluster"] = list_Cluster

df_cluster_kmeans_31_fam = df_elements_by_famliy.copy()
list_Cluster = []
for fam in df_cluster_kmeans_31_fam["Family"]:
    cluster = df_profundidades_family_kmeans_31[df_profundidades_family_kmeans_31["Family"] == fam]["Cluster"].values[0]
    list_Cluster.append(cluster)
df_cluster_kmeans_31_fam["Cluster"] = list_Cluster

# Genero
df_cluster_kmeans_15_genus = df_elements_by_genus.copy()
list_Cluster = []
for fam in df_cluster_kmeans_15_genus["Genus"]:
    cluster = df_profundidades_genero_kmeans_15[df_profundidades_genero_kmeans_15["Genus"] == fam]["Cluster"].values[0]
    list_Cluster.append(cluster)
df_cluster_kmeans_15_genus["Cluster"] = list_Cluster

df_cluster_kmeans_31_genus = df_elements_by_genus.copy()
list_Cluster = []
for fam in df_cluster_kmeans_31_genus["Genus"]:
    cluster = df_profundidades_genero_kmeans_31[df_profundidades_genero_kmeans_31["Genus"] == fam]["Cluster"].values[0]
    list_Cluster.append(cluster)
df_cluster_kmeans_31_genus["Cluster"] = list_Cluster

In [90]:
df_Cluster_Families_15_kmeans = pd.DataFrame(columns=["Cluster", "Familias"])
for clus in df_cluster_kmeans_15_fam["Cluster"].unique():
    famsList = ""
    for fam in df_cluster_kmeans_15_fam[df_cluster_kmeans_15_fam["Cluster"] == clus]["Family"].unique():
        famsList = famsList + ", " + fam
    df_Cluster_Families_15_kmeans.loc[len(df_Cluster_Families_15_kmeans.index)] = [clus, famsList] 

df_Cluster_Families_15_kmeans = df_Cluster_Families_15_kmeans.sort_values(by="Cluster")

df_Cluster_Families_31_kmeans = pd.DataFrame(columns=["Cluster", "Familias"])
for clus in df_cluster_kmeans_31_fam["Cluster"].unique():
    famsList = ""
    for fam in df_cluster_kmeans_31_fam[df_cluster_kmeans_31_fam["Cluster"] == clus]["Family"].unique():
        famsList = famsList + ", " + fam
    df_Cluster_Families_31_kmeans.loc[len(df_Cluster_Families_31_kmeans.index)] = [clus, famsList] 

df_Cluster_Families_31_kmeans = df_Cluster_Families_31_kmeans.sort_values(by="Cluster")

df_Cluster_Genero_15_kmeans = pd.DataFrame(columns=["Cluster", "Genero"])
for clus in df_cluster_kmeans_15_genus["Cluster"].unique():
    famsList = ""
    for fam in df_cluster_kmeans_15_genus[df_cluster_kmeans_15_genus["Cluster"] == clus]["Genus"].unique():
        famsList = famsList + ", " + fam
    df_Cluster_Genero_15_kmeans.loc[len(df_Cluster_Genero_15_kmeans.index)] = [clus, famsList] 

df_Cluster_Genero_15_kmeans = df_Cluster_Genero_15_kmeans.sort_values(by="Cluster")

df_Cluster_Genero_31_kmeans = pd.DataFrame(columns=["Cluster", "Genero"])
for clus in df_cluster_kmeans_15_genus["Cluster"].unique():
    famsList = ""
    for fam in df_cluster_kmeans_15_genus[df_cluster_kmeans_15_genus["Cluster"] == clus]["Genus"].unique():
        famsList = famsList + ", " + fam
    df_Cluster_Genero_31_kmeans.loc[len(df_Cluster_Genero_31_kmeans.index)] = [clus, famsList] 

df_Cluster_Genero_31_kmeans = df_Cluster_Genero_31_kmeans.sort_values(by="Cluster")

#### Metodo 3 - DBSCAN

El algoritmo DBSCAN funciona de la siguiente manera:

1. Selecciona una muestra aleatoria y examina su vecindario mediante una medida de distancia especificada.
2. Si el número de muestras dentro del vecindario, incluida la muestra en sí misma, supera un umbral de densidad predefinido, se considera un "núcleo".
3. Expande el grupo de manera iterativa agregando todas las muestras alcanzables desde el núcleo, es decir, todas las muestras dentro del umbral de densidad.
4. Repite los pasos anteriores hasta que se haya examinado todas las muestras.

Bibliografía:

- Ester, M., Kriegel, H. P., Sander, J., & Xu, X. (1996). A density-based algorithm for discovering clusters in large spatial databases with noise. En KDD (Vol. 96, No. 34, pp. 226-231).
- Schubert, E., Sander, J., Ester, M., Kriegel, H. P., & Xu, X. (2017). DBSCAN revisited, revisited: why and how you should (still) use DBSCAN. ACM Transactions on Database Systems (TODS), 42(3), 19.
- Gan, G., Ma, C., & Wu, J. (2007). Data clustering: theory, algorithms, and applications. SIAM.
- Zhou, J., Sun, X., Zha, H., & Huang, Z. (2018). A brief survey on DBSCAN: from the algorithm to variations and applications. ACM Transactions on Intelligent Systems and Technology (TIST), 9(5), 1-29.

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html

In [280]:
df_features_3_15 = df_elements_by_famliy.copy()
df_features_3_15["Family_Numeric"] = pd.factorize(df_features_3_15["Family"])[0]
df_features_3_15 = df_features_3_15.drop(columns='Family')
# Instantiate the DBSCAN model with the desired parameters
dbscan = DBSCAN(eps=4, min_samples=3)

# Fit the model to the data
dbscan.fit(df_features_3_15)

# Add the cluster labels to the dataframe
df_features_3_15['Cluster'] = dbscan.labels_
df_features_3_15["Family"] = df_elements_by_famliy["Family"]
# --------------------------------------------------------------
df_features_3_31 = df_elements_by_famliy.copy()
df_features_3_31["Family_Numeric"] = pd.factorize(df_features_3_31["Family"])[0]
df_features_3_31 = df_features_3_31.drop(columns='Family')
# Instantiate the DBSCAN model with the desired parameters
dbscan = DBSCAN(eps=0.5, min_samples=31)

# Fit the model to the data
dbscan.fit(df_features_3_31)

# Add the cluster labels to the dataframe
df_features_3_31['Cluster'] = dbscan.labels_
df_features_3_31["Family"] = df_elements_by_famliy["Family"]
# --------------------------------------------------------------
df_features_3_62 = df_elements_by_famliy.copy()
df_features_3_62["Family_Numeric"] = pd.factorize(df_features_3_62["Family"])[0]
df_features_3_62 = df_features_3_62.drop(columns='Family')
# Instantiate the DBSCAN model with the desired parameters
dbscan = DBSCAN(eps=0.5, min_samples=62)

# Fit the model to the data
dbscan.fit(df_features_3_62)

# Add the cluster labels to the dataframe
df_features_3_62['Cluster'] = dbscan.labels_
df_features_3_62["Family"] = df_elements_by_famliy["Family"]

###  ¿Cual es la diferencia entre los diferentes clustes?

<p>Cada uno tiene su caracteristicas y mejores casos de usos, talvez aqui no sea el mejor problema para ser utilizado 1, 2 o de ellos, pero sirve para comprender su funcionalidad.</p>

![](Cluster_1.png)

![](Cluster_2.png)

https://medium.com/@namanbhandari/agglomerative-clustering-vs-k-means-clustering-766a90b37dc0

### Visualización
---

<h4> Metodo 1 - Cluster agglomerative </h4>

In [75]:
fig = px.line(df_cluster_agg_15_fam, x="Profundidad", y="Valor", color='Cluster')
fig.show()

In [70]:
df_Cluster_Families_15

,Cluster,Familias
2,0,", Asparagaceae, Betulaceae, Acanthaceae, Ericaceae, Berberidaceae, Burseraceae, Cactaceae"
13,1,", Urticaceae, Zygophyllaceae, Pteridaceae, Anemiaceae, Ophioglossaceae, Dryopteridaceae, Lygodiaceae, Lycopodiaceae, Osmundaceae, Selaginellaceae"
11,2,", Onagraceae, Rubiaceae, Monocotyledoneae, Hydrophyllaceae, Plantaginaceae, Polemoniaceae, Rhamnaceae, Solanaceae, Typhaceae"
10,3,", Cyperaceae"
5,4,", Rutaceae, Adoxaceae, Bignoniaceae, Nyctaginaceae"
3,5,", Celastraceae, Malvaceae, Cannabaceae, Salicaceae, Sapindaceae, Ephedraceae, Oleaceae, Juglandaceae, Cupressaceae, Rosaceae"
9,6,", Euphorbiaceae, Polygonaceae, Gentianaceae, Lamiaceae, Verbenaceae, Lythraceae"
0,7,", Pinaceae"
7,8,", Asteraceae"
6,9,", Amaranthaceae"


In [77]:
fig = px.line(df_cluster_agg_31_fam, x="Profundidad", y="Valor", color='Cluster')
fig.show()

In [78]:
df_Cluster_Families_31

,Cluster,Familias
5,0,", Celastraceae, Malvaceae"
2,1,", Asparagaceae, Acanthaceae, Ericaceae, Berberidaceae, Burseraceae"
23,2,", Onagraceae, Rubiaceae, Monocotyledoneae, Hydrophyllaceae, Plantaginaceae"
10,3,", Rutaceae, Adoxaceae, Bignoniaceae"
27,4,", Urticaceae, Zygophyllaceae, Pteridaceae, Anemiaceae, Ophioglossaceae, Dryopteridaceae"
8,5,", Juglandaceae, Cupressaceae, Rosaceae"
29,6,", Lycopodiaceae, Osmundaceae, Selaginellaceae"
19,7,", Cyperaceae"
26,8,", Typhaceae"
14,9,", Apiaceae, Brassicaceae"


#### Method 2 - KMeans

In [88]:
fig = px.line(df_cluster_kmeans_15_fam, x="Profundidad", y="Valor", color='Cluster', title="KMeans Cluster (15) - Familia")
fig.show()

In [79]:
df_Cluster_Genero_15

,Cluster,Genero
14,0,", Urtica, Zea, Adiantum, Anemia, Botrychium, Cheilanthes, Gymnocarpium, Lygodium, Monoleta, Ophioglossum, Osmunda, Selaginella, Pellaea"
12,1,", Lythrum, Manihot, Marina, Mitracarpus, Nama, Oenothera, Plantago, Polygonum, Sida, Sphaeralcea, Tetramerium, Trixis"
3,2,", Desmanthus, Dodonaea, Ephedra, Fraxinus, Juglans, Juniperus, Mimosa, Opuntia, Parkinsonia, Picea"
10,3,", Boerhavia, Callaeum, Cirsium, Clematis, Croton, Dalea, Dicliptera"
9,4,", Artemisia"
11,5,", Eriogonum, Eustoma, Evolvulus, Hibiscus, Ipomoea, Justicia, Lippia, Lotus"
6,6,", Prunus, Salix, Sambucus, Tecoma, Allionia"
1,7,", Unknown"
2,8,", Canotia, Ceiba, Celtis, Populus, Cylindropuntia, Dasylirion"
4,9,", Pinus"


In [89]:
fig = px.line(df_cluster_kmeans_31_fam, x="Profundidad", y="Valor", color='Cluster', title="KMeans Cluster (31) - Familia")
fig.show()